In [1]:
#Imports
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
import time
import os
import csv
import json
import requests
from webdriver_manager.chrome import ChromeDriverManager



In [2]:
#DIRECT FLIGHT URL
url = "https://flysanantonio.com/home/flights/nonstop-destinations/"

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Downloading: 100%|█████████████████| 8.08M/8.08M [00:01<00:00, 5.25MB/s]


In [4]:
#Direct Flight WebScraping

browser.visit(url)
time.sleep(1)

airportNames = []
airportCodes = []
airportList = []
airportDict = {}


html = browser.html
soup = BeautifulSoup(html, 'html.parser')

airportName = soup.find_all('span', class_="title")
airportCode = soup.find_all('span', class_='airport-code')
    
for (title, code) in zip(airportName, airportCode):
    airportDict = {}
    airportDict['Title'] = title.text
    airportDict['Code'] = code.text
    airportList.append(airportDict)

In [5]:
#Direct Flights
airportList
airport_df = pd.DataFrame(airportList)
airport_df



,Title,Code
0,"Albuquerque, NM",ABQ
1,"Atlanta, GA",ATL
2,"Baltimore, MD",BWI
3,"Boston, MA",BOS
4,"Cancun, Mexico",CUN
5,"Charlotte, NC",CLT
6,"Chicago-O’Hare, IL",ORD
7,"Chicago-Midway, IL",MDW
8,"Colorado Springs, CO",COS
9,"Dallas-Love, TX",DAL


In [6]:
#Prepping API request for Direct Flights 
code_list = airport_df['Code'].tolist()
url2 = "https://aviation-edge.com/v2/public/airportDatabase?key=cf86e1-a6b14f&codeIataAirport="
response_json = []


In [7]:
# Make a request for each of airport codes
for x in range(len(code_list)):

    # Get one of the posts
    post_response = requests.get(url2+ code_list[x])

    # Save post's JSON
    response_json.append(post_response.json())

In [8]:
response_json

[[{'GMT': '-7',
   'airportId': 41,
   'codeIataAirport': 'ABQ',
   'codeIataCity': 'ABQ',
   'codeIcaoAirport': 'KABQ',
   'codeIso2Country': 'US',
   'geonameId': '5454724',
   'latitudeAirport': 35.049625,
   'longitudeAirport': -106.617195,
   'nameAirport': 'Albuquerque International',
   'nameCountry': 'United States',
   'phone': '505-244-7700',
   'timezone': 'America/Denver'}],
 [{'GMT': '-5',
   'airportId': 411,
   'codeIataAirport': 'ATL',
   'codeIataCity': 'ATL',
   'codeIcaoAirport': 'KATL',
   'codeIso2Country': 'US',
   'geonameId': '4180439',
   'latitudeAirport': 33.640068,
   'longitudeAirport': -84.44403,
   'nameAirport': 'Hartsfield-jackson Atlanta International',
   'nameCountry': 'United States',
   'phone': '800-897-1910',
   'timezone': 'America/New_York'}],
 [{'GMT': '-5',
   'airportId': 1043,
   'codeIataAirport': 'BWI',
   'codeIataCity': 'BWI',
   'codeIcaoAirport': 'KBWI',
   'codeIso2Country': 'US',
   'geonameId': '4347822',
   'latitudeAirport': 39.1

In [9]:
#Direct Flight Weather API
url3 = "http://api.openweathermap.org/data/2.5/weather?appid=53a798ea46d77e04aa2aeafa156ccdea&units=imperial&q="

cities = ["Albuquerque", "Atlanta", "Baltimore", "Boston", "Cancun",
"Charlotte", "Chicago", "Colorado Springs", "Dallas","Fort Worth","Denver", "Detroit", "El Paso", "Guadalajara", "Houston", "Houston", "Kansas City", "Las Vegas",
"Leon", "Los Angeles" ,"Mexico City", "Miami", "Minneapolis", "Monterrey", "Nashville", "New Orleans","New York City", "New York City",
"Oklahoma City","Orlando", "Philadelphia","Phoenix","Salt Lake City", "San Diego","San Francisco","Seattle",
"St. Louis", "Tampa", "Washington DC"]

temp = []

# Loop through the list of cities and perform a request for data on each
for city in cities:
    try:
        response = requests.get(url3 + city).json()
        temp.append(response['main']['temp'])
       
    
    except: 
        print("Weather not found")
        pass

print(temp)



[61.68, 64.15, 63.88, 64.17, 85.93, 69.1, 50.31, 54.23, 70.93, 69.44, 52.9, 51.01, 67.06, 67.98, 79.84, 79.84, 58.35, 72.97, 77.14, 65.86, 64.31, 86.22, 41.58, 86.77, 60.3, 77.43, 67.55, 67.55, 61.12, 81.72, 66.52, 78.87, 58.68, 66.87, 55.38, 51.51, 57.09, 80.83, 64.36]


In [10]:
hotel_csv = "hotels.csv"

hotel_df = pd.read_csv(hotel_csv )
hotel_df

,Unnamed: 0,Hotel Name,Rating,Address,City Code,Place ID
0,0,"Holiday Inn & Suites Albuquerque Airport, an I...",3.7,"1501 Sunport Place Southeast, Albuquerque",ABQ,ChIJWSNsYHcMIocRAEwTzmc2p4Q
1,1,Hilton Garden Inn Albuquerque Airport,3.9,"2601 Yale Boulevard Southeast, Albuquerque",ABQ,ChIJ19wg04ULIocRD295rgmyS5o
2,2,Holiday Inn Express & Suites Albuquerque Airpo...,4.3,"1921 Yale Boulevard Southeast, Albuquerque",ABQ,ChIJKxw9BYILIocRxWmgRIN0608
3,3,Sheraton Albuquerque Airport Hotel,4.0,"2910 Yale Boulevard Southeast, Albuquerque",ABQ,ChIJO1Sk9IULIocROm7jBp0RNYQ
4,4,Fairfield Inn & Suites by Marriott Albuquerque...,3.9,"2300 Centre Avenue Southeast, Albuquerque",ABQ,ChIJ13wBU4ILIocRpT2xkae5lMY
...,...,...,...,...,...,...
182,182,Washington Dulles Airport Marriott,4.0,"45020 Aviation Drive, Dulles",WAS,ChIJK87f1INHtokR8V-p_sIlSCc
183,183,The Westin Washington Dulles Airport,4.2,"2520 Wasser Terrace, Herndon",WAS,ChIJ3ZQHkXZHtokR0exa1cY04BY
184,184,"Candlewood Suites Washington-Dulles Herndon, a...",4.0,"13845 Sunrise Valley Drive, Herndon",WAS,ChIJIY5UmHVHtokRbH1F9nyGPFY
185,185,Hyatt Regency Dulles,4.2,"2300 Dulles Corner Boulevard, Herndon",WAS,ChIJQUumvYJHtokRz1JgP9NRqWg


In [11]:
#Export to CSV
airport_df["Temperature_in_F"]= [75.96, 63.73, 63.86, 60.64, 82.33, 64.27, 52.41, 60.78, 85.96, 84.15, 57.07, 65.7, 78.94, 75.97, 80.73, 80.73, 65.48, 88.79, 75.25, 71.69, 73.44, 82.6, 45.23, 84.81, 62.91, 77.14, 61.97, 61.97, 70.75, 75.18, 61.83, 93.07, 68.95, 70.43, 61.9, 65.5, 63.48, 78.78, 65.66,70.00]
airport_df.to_csv("direct_flights2.csv")

In [12]:
#Indirect Flights
file = "indirect_airports.csv"

indirect_df= pd.read_csv(file)
indirect_df

,airport_name,latitude_deg,longitude_deg,iso_region,municipality,iata_code
0,Fort Lauderdale Hollywood International Airport,26.072599,-80.152702,US-FL,Fort Lauderdale,FLL
1,Daniel K Inouye International Airport,21.320620,-157.924228,US-HI,Honolulu,HNL
2,Portland International Airport,45.588699,-122.598000,US-OR,Portland,PDX
3,Tampa International Airport,27.975500,-82.533203,US-FL,Tampa,TPA
4,Cincinnati Northern Kentucky International Air...,39.048801,-84.667801,US-KY,Cincinnati,CVG
...,...,...,...,...,...,...
60,Rapid City Regional Airport,44.045300,-103.056999,US-SD,Rapid City,RAP
61,McGhee Tyson Airport,35.811001,-83.994003,US-TN,Knoxville,TYS
62,Tallahassee Regional Airport,30.396500,-84.350304,US-FL,Tallahassee,TLH
63,Fresno Yosemite International Airport,36.776199,-119.718002,US-CA,Fresno,FAT


In [13]:
#Indirect Flight Weather API
cities2 = ['Fort Lauderdale','Honolulu','Portland','Tampa','Cincinnati','Oakland','Pittsburgh','San Jose','Raleigh','Memphis',
 'Santa Ana','Cleveland','Indianapolis','Columbus','Tucson','Buffalo','Anchorage','Jacksonville','Hartford','West Palm Beach','Kahului','Sacramento','Norfolk','Reno','Burbank','Long Beach','Providence','Oshkosh','Omaha','Louisville','Ontario','Syracuse','White Plains','Albany','Charleston','Birmingham','Richmond','Rochester','Savannah','Dayton','Spokane','Boise','Manchester',
 'Fort Myers','Portland','Santa Barbara','Wichita','Palm Springs','Madison','Grand Rapids','Des Moines','Burlington','Little Rock','Greensboro','Tulsa','Kailua','Santa Monica','Pensacola','Teterboro','Van Nuys','Rapid City','Knoxville','Tallahassee','Fresno','Sarasota']
temp2= []

# Loop through the list of cities and perform a request for data on each
for city2 in cities2:
    try:
        response2 = requests.get(url3 + city2).json()
        temp2.append(response2['main']['temp'])
       
    
    except: 
        print("Weather not found")
        pass

    
print(temp2)

[86.79, 74.55, 58.17, 80.83, 55.4, 49.42, 52.16, 57.79, 69.66, 61.92, 65.91, 53.73, 52.97, 54.23, 72.88, 51.06, 27.63, 78.84, 67.64, 84.22, 78.12, 56.86, 62.44, 54.66, 64.94, 66.16, 62.96, 42.31, 52.74, 58.37, 63.37, 69.17, 67.78, 63.07, 75.42, 66.51, 67.1, 56.62, 72.09, 54.9, 43.47, 53.87, 58.48, 77.86, 58.17, 53.65, 61, 80.85, 64.9, 47.88, 49.32, 67.73, 66.09, 67.64, 63.57, 74.14, 66.42, 76.15, 67.1, 64.78, 49.41, 65.88, 78.67, 64.11, 78.78]


In [14]:
#Export to CSV
indirect_df["Temperature_in_F"]= [85.93, 76.14, 49.87, 79.84, 60.48, 59.5, 59.25, 56.68, 64.69, 67.12, 67.12, 63.77, 61.92, 60.94, 67.66, 63.16, 30.42, 77.05, 59.02, 85.77, 75.22, 58.26, 57.51, 52.88, 65.35, 67.21, 60.3, 60.78, 65.1, 61.47, 64.54, 76.42, 60.24, 52.36, 74.53, 64.62, 63.45, 58.01, 73.83, 62.64, 50.99, 49.89, 58.17, 83.39, 49.82, 51.73, 64.92, 75.85, 62.91, 61.36, 64.71, 62.73, 69.55, 59.49, 65.26, 75.9, 66.83, 70.45, 61.95, 64.71, 64.58, 58.42, 69.64, 63.86, 83.44]
indirect_df.to_csv("indirect_final.csv")

In [20]:
code2_list = indirect_df['iata_code'].tolist()
response_json2 = []


In [21]:
# Make a request for each of airport codes
for x in range(len(code2_list)):

    # Get one of the posts
    post_response2 = requests.get(url2+ code2_list[x])

    # Save post's JSON
    response_json2.append(post_response2.json())

In [22]:
response_json2

[[{'GMT': '-5',
   'airportId': 2344,
   'codeIataAirport': 'FLL',
   'codeIataCity': 'FLL',
   'codeIcaoAirport': 'KFLL',
   'codeIso2Country': 'US',
   'geonameId': '4155976',
   'latitudeAirport': 26.071491,
   'longitudeAirport': -80.144905,
   'nameAirport': 'Fort Lauderdale–Hollywood International',
   'nameCountry': 'United States',
   'phone': '',
   'timezone': 'America/New_York'}],
 [{'GMT': '-10',
   'airportId': 3014,
   'codeIataAirport': 'HNL',
   'codeIataCity': 'HNL',
   'codeIcaoAirport': 'PHNL',
   'codeIso2Country': 'US',
   'geonameId': '5856213',
   'latitudeAirport': 21.325832,
   'longitudeAirport': -157.92166,
   'nameAirport': 'Honolulu International',
   'nameCountry': 'United States',
   'phone': '808-836-6411',
   'timezone': 'Pacific/Honolulu'}],
 [{'GMT': '-8',
   'airportId': 6023,
   'codeIataAirport': 'PDX',
   'codeIataCity': 'PDX',
   'codeIcaoAirport': 'KPDX',
   'codeIso2Country': 'US',
   'geonameId': '5746563',
   'latitudeAirport': 45.588997,
   

In [24]:
with open('airports2.json', 'w') as json_file:
    json.dump(response_json2, json_file)